In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

base_dir = r'E:\Khóa luận\Data\Phase 2'
export_plot_dir = r'E:\Khóa luận\Exports\Processed'
os.makedirs(export_plot_dir, exist_ok=True)

input_files = {
    '250500': os.path.join(base_dir, '250500.csv'),
    '500500': os.path.join(base_dir, '500500.csv'),
    '750500': os.path.join(base_dir, '750500.csv')
}

for label, file_path in input_files.items():
    print(f"\n===== Đang xử lý nồng độ {label} =====")
    df = pd.read_csv(file_path, sep=';', encoding='utf-8', low_memory=False)
    print(f"Số dòng ban đầu: {len(df)}")

    # 1. Kiểm tra trùng lặp, kiểu dữ liệu không nhất quán, giá trị không thể
    df = df.drop_duplicates()
    print(f"Số dòng sau khi loại trùng lặp: {len(df)}")
    # Chuyển các cột số sang numeric, cột thời gian sang datetime
    if 'Date time' in df.columns:
        df['Date time'] = pd.to_datetime(df['Date time'], errors='coerce')
    numerical_cols = [col for col in df.columns if col not in ['Timestamp', 'Date time'] and pd.api.types.is_numeric_dtype(pd.to_numeric(df[col], errors='coerce'))]
    for col in numerical_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Loại bỏ giá trị âm
        df.loc[df[col] < 0, col] = np.nan

    # 2. Xử lý giá trị thiếu, toàn 0
    nan_counts = df.isna().sum()
    zero_counts = (df[numerical_cols] == 0).sum()
    # Điền thiếu (nội suy, ffill, bfill, median)
    for col in numerical_cols:
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        df[col] = df[col].ffill().bfill()
        df[col] = df[col].fillna(df[col].median())
    # Loại bỏ cột có >50% giá trị thiếu/toàn 0
    threshold = 0.5
    cols_to_drop = []
    for col in numerical_cols:
        missing_ratio = nan_counts[col] / len(df)
        zero_ratio = (df[col] == 0).sum() / len(df)
        if missing_ratio > threshold or zero_ratio > threshold:
            cols_to_drop.append(col)
    if cols_to_drop:
        print(f"Các cột bị loại bỏ do quá nhiều thiếu/0: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
        numerical_cols = [col for col in numerical_cols if col not in cols_to_drop]

    # 3. Xử lý ngoại lai (outlier) bằng IQR
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df.loc[(df[col] < lower) | (df[col] > upper), col] = np.nan
        # Điền lại giá trị thiếu sau khi loại outlier
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        df[col] = df[col].ffill().bfill()
        df[col] = df[col].fillna(df[col].median())

    # 4. Biến đổi thành stationary (differencing)
    for col in numerical_cols:
        df[col] = df[col].diff().fillna(0)

    # 5. Loại bỏ cột toàn 0/toàn NaN sau differencing
    cols_all_zero_or_nan = [col for col in numerical_cols if df[col].isna().all() or (df[col] == 0).all()]
    if cols_all_zero_or_nan:
        print(f"Các cột bị loại bỏ sau differencing: {cols_all_zero_or_nan}")
        df = df.drop(columns=cols_all_zero_or_nan)
        numerical_cols = [col for col in numerical_cols if col not in cols_all_zero_or_nan]

    # 6. Chuẩn hóa dữ liệu
    if len(numerical_cols) > 0:
        scaler = StandardScaler()
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    # 7. Lưu dữ liệu đã xử lý
    export_path = os.path.join(export_plot_dir, f"{label}_processed.csv")
    df.to_csv(export_path, index=False)
    print(f"Đã xuất dữ liệu đã tiền xử lý ra: {export_path}")

    # ========== TRỰC QUAN HÓA ==========
    export_dir = export_plot_dir  # dùng chung thư mục export

    # Violinplot
    if len(numerical_cols) > 0:
        plt.figure(figsize=(max(10, len(numerical_cols)*1.2), 6))
        sns.violinplot(data=df[numerical_cols], inner='box', cut=0)
        plt.title(f'Violinplot tất cả biến số - Nồng độ {label}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'violinplot_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ violinplot cho nồng độ {label}.")

    # Boxplot
    if len(numerical_cols) > 0:
        plt.figure(figsize=(max(10, len(numerical_cols)*1.2), 6))
        sns.boxplot(data=df[numerical_cols])
        plt.title(f'Boxplot tất cả biến số - Nồng độ {label}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'boxplot_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ boxplot cho nồng độ {label}.")

    # Histogram (subplots)
    if len(numerical_cols) > 0:
        n_cols = 4
        n_rows = int(np.ceil(len(numerical_cols)/n_cols))
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3))
        axes = axes.flatten()
        for i, col in enumerate(numerical_cols):
            sns.histplot(df[col], bins=30, kde=True, ax=axes[i], color='steelblue')
            axes[i].set_title(col)
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        plt.suptitle(f'Histogram các biến số - Nồng độ {label}', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.97])
        plt.savefig(os.path.join(export_dir, f'histogram_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ histogram cho nồng độ {label}.")

    # Heatmap tương quan
    if len(numerical_cols) > 1:
        plt.figure(figsize=(max(8, len(numerical_cols)), max(6, len(numerical_cols)*0.6)))
        corr = df[numerical_cols].corr()
        sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
        plt.title(f'Heatmap ma trận tương quan - Nồng độ {label}')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'corr_heatmap_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không đủ biến số để vẽ heatmap tương quan cho nồng độ {label}.")

    # Biểu đồ missing/zero barplot
    if len(numerical_cols) > 0:
        nan_counts = df[numerical_cols].isna().sum()
        zero_counts = (df[numerical_cols] == 0).sum()
        missing_df = pd.DataFrame({
            'missing': nan_counts,
            'zero': zero_counts
        })
        missing_df['total'] = missing_df['missing'] + missing_df['zero']
        missing_df = missing_df.sort_values('total', ascending=False)
        plt.figure(figsize=(max(8, len(numerical_cols)*0.7), 5))
        missing_df[['missing', 'zero']].plot(kind='bar', stacked=True, ax=plt.gca(), color=['#FFA07A', '#87CEFA'])
        plt.title(f'Số lượng giá trị thiếu và bằng 0 trên từng biến số - Nồng độ {label}')
        plt.ylabel('Số lượng')
        plt.xlabel('Tên biến')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'missing_and_zero_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ missing/zero barplot cho nồng độ {label}.")

    # Time series plot cho 3 biến đầu tiên (nếu có Date time)
    if 'Date time' in df.columns and len(numerical_cols) > 0:
        for col in numerical_cols[:3]:
            if df[col].nunique(dropna=True) < 2 or df[col].dropna().empty:
                continue
            plt.figure(figsize=(12, 4))
            plt.plot(df['Date time'], df[col], label=col)
            plt.title(f'Chuỗi thời gian của {col} - Nồng độ {label}')
            plt.xlabel('Thời gian')
            plt.ylabel(col)
            plt.tight_layout()
            plt.savefig(os.path.join(export_dir, f'timeseries_{col}_{label}.png'))
            plt.close()
    elif 'Date time' in df.columns:
        print(f"[CẢNH BÁO] Không có biến số nào hợp lệ để vẽ time series cho nồng độ {label}.")

print(f"\nĐã xử lý và lưu toàn bộ biểu đồ cho các nồng độ vào: {export_plot_dir}")


===== Đang xử lý nồng độ 250500 =====
Số dòng ban đầu: 11839
Số dòng sau khi loại trùng lặp: 11839
Các cột bị loại bỏ do quá nhiều thiếu/0: ['H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\250500_processed.csv

===== Đang xử lý nồng độ 500500 =====
Số dòng ban đầu: 8646
Số dòng sau khi loại trùng lặp: 8646
Các cột bị loại bỏ do quá nhiều thiếu/0: ['ccsTVOC', 'H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Các cột bị loại bỏ sau differencing: ['ccseCO2']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\500500_processed.csv

===== Đang xử lý nồng độ 750500 =====
Số dòng ban đầu: 26315
Số dòng sau khi loại trùng lặp: 26315
Các cột bị loại bỏ do quá nhiều thiếu/0: ['ccsTVOC', 'H2S_concentration', 'CH4ppm', 'HCHOmg/m3', 'TVOCppm']
Các cột bị loại bỏ sau differencing: ['ccseCO2']
Đã xuất dữ liệu đã tiền xử lý ra: E:\Khóa luận\Exports\Processed\750500_processed.csv

Đã xử lý và lưu toàn bộ biểu đồ 